# 📊 Manufacturing ABAC Demo - Step 3: Setup Governed Tags

## 📋 Overview
This notebook creates the tag policies defining the necessary governed tags and their allowed values

### What This Notebook Does:
1. **Creates Tag policy**: Uses REST API to create tag policies defining governed tags and their allowed values
2. **Demonstrates Usage**: Shows how to apply tags to relevant columns in the manufacturing datasets

## 🎓 How to Use This Notebook
1. **Ensure Steps 1-2 Complete**: All functions, tables, and data must exist
2. **Run All Cells**: Execute sequentially to see all test results
3. **Verify Expectations**: Check that tags have been applied through the catalog explorer

## ⚙️ Prerequisites
- ✅ **Step 1 completed**: All functions created
- ✅ **Step 2 completed**: Core tables with data
- ✅ APPLY TAG permission on all tables

## 📊 Expected Results
After running this notebook:
- ✅ Tag policies are created
- ✅ Governed tags assigned
- ✅ Prepared for creating ABAC policies


## ⚙️ Configuration

Using the same catalog and schema from previous steps:
- **Catalog**: `your_catalog_name`
- **Schema**: `manufacturing`


In [0]:
pip install pyyaml

In [0]:
# 📋 Load Configuration from config.yaml
import yaml
from pathlib import Path
import requests
import os
from databricks.sdk import WorkspaceClient

config_file = Path('config.yaml')
if config_file.exists():
    with open(config_file) as f:
        config = yaml.safe_load(f)
    CATALOG = config['catalog']
    SCHEMA = config['schema']
    print(f'✅ Configuration loaded from config.yaml')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')
else:
    # Fallback defaults
    CATALOG = 'your_catalog_name'
    SCHEMA = 'manufacturing'
    print(f'⚠️  config.yaml not found - using defaults')
    print(f'   📊 Catalog: {CATALOG}')
    print(f'   📁 Schema: {SCHEMA}')

# Set catalog and schema to use for the cells below
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA}")
spark.sql(f"USE SCHEMA {SCHEMA}")

client = WorkspaceClient()
workspace_url = client.config.host

In [0]:
%sql
SELECT '🎯 Target: ' || current_catalog() || '.' || current_schema() AS status;

###Define Governed Tags + Allowed Values
https://docs.databricks.com/api/workspace/tagpolicies/createtagpolicy

In [0]:
def get_token():
    ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
    return getattr(ctx, "apiToken")().get()


def create_tag_policy(payload):
    data = requests.post(
        f"{workspace_url}/api/2.1/tag-policies",
        headers={"Authorization": f"Bearer {get_token()}"},
        json=payload,
    )

    return data

In [0]:
manufacturing_tag_policies = {
    "supplier_scope": {"values": [
        "","",""
    ], "description": ""}
}

In [0]:
sensitive_payload = {
    "description": "Manufacturing sensitive data types",
    "tag_key": "sensitive_type_manufacturing",
    "values": [
        {"name": "name"},
        {"name": "gps"},
        {"name": "cost"},
        {"name": "email"},
        {"name": "phone"},
        {"name": "serial_number"},
        {"name": "specification"},
        {"name": "ts"},
        {"name": "CAD"}
    ]
}
print(create_tag_policy(sensitive_payload).json())

data_purpose_payload = {
    "description": "Intended use purpose for data access",
    "tag_key": "data_purpose_manufacturing",
    "values": [
        {"name": "Operations"},
        {"name": "Quality"},
        {"name": "Maintenance"},
        {"name": "SupplyChain"},
        {"name": "Audit"},
        {"name": "RnD"}
    ]
}
print(create_tag_policy(data_purpose_payload).json())

shift_hours_payload = {
    "description": "Shift-based access control",
    "tag_key": "shift_hours_manufacturing",
    "values": [
        {"name": "Day"},
        {"name": "Swing"},
        {"name": "Night"},
        {"name": "Emergency_24x7"}
    ]
}
print(create_tag_policy(shift_hours_payload).json())

asset_criticality_payload = {
    "description": "Asset criticality based access",
    "tag_key": "asset_criticality_manufacturing",
    "values": [
        {"name": "High"},
        {"name": "Medium"},
        {"name": "Low"}
    ]
}
print(create_tag_policy(asset_criticality_payload).json())

export_control_payload = {
    "description": "Export control classification",
    "tag_key": "export_control_manufacturing",
    "values": [
        {"name": "ITAR"},
        {"name": "EAR99"},
        {"name": "Not_Controlled"}
    ]
}
print(create_tag_policy(export_control_payload).json())

ip_sensitivity_payload = {
    "description": "Design/IP sensitivity classification",
    "tag_key": "ip_sensitivity_manufacturing",
    "values": [
        {"name": "Trade_Secret"},
        {"name": "Internal"},
        {"name": "Public"}
    ]
}
print(create_tag_policy(ip_sensitivity_payload).json())

data_residency_payload = {
    "description": "Geographic data residency requirements",
    "tag_key": "data_residency_manufacturing",
    "values": [
        {"name": "Country_Only"},
        {"name": "EU_Only"},
        {"name": "Cross_Border_Approved"}
    ]
}
print(create_tag_policy(data_residency_payload).json())

supplier_scope_payload = {
    "description": "Scope restriction for supplier data",
    "tag_key": "supplier_scope_manufacturing",
    "values": [
        {"name": "Named_Supplier_Only"},
        {"name": "All_Suppliers"},
        {"name": "Audit_Project_Only"}
    ]
}
print(create_tag_policy(supplier_scope_payload).json())

In [0]:
%sql
ALTER TABLE maintenance_events ALTER COLUMN technician_email SET TAGS ('ip_sensitivity_manufacturing' = 'Internal','sensitive_type_manufacturing' = 'email');
ALTER TABLE maintenance_events ALTER COLUMN technician_phone SET TAGS ('ip_sensitivity_manufacturing' = 'Internal','sensitive_type_manufacturing' = 'phone');
ALTER TABLE maintenance_events ALTER COLUMN cost SET TAGS ('data_purpose_manufacturing' = 'SupplyChain','sensitive_type_manufacturing' = 'cost');
ALTER TABLE maintenance_events ALTER COLUMN start_time SET TAGS ('asset_criticality_manufacturing' = 'High','sensitive_type_manufacturing' = 'ts');
ALTER TABLE maintenance_events ALTER COLUMN end_time SET TAGS ('asset_criticality_manufacturing' = 'High','sensitive_type_manufacturing' = 'ts');

In [0]:
%sql
ALTER TABLE product_specs ALTER COLUMN spec_text SET TAGS ('ip_sensitivity_manufacturing' = 'Trade_Secret','sensitive_type_manufacturing' = 'specification');
ALTER TABLE product_specs ALTER COLUMN cad_file_uri SET TAGS ('ip_sensitivity_manufacturing' = 'Trade_Secret','sensitive_type_manufacturing' = 'CAD');
ALTER TABLE product_specs ALTER COLUMN material_composition SET TAGS ('ip_sensitivity_manufacturing' = 'Internal');
ALTER TABLE product_specs ALTER COLUMN created_by SET TAGS ('ip_sensitivity_manufacturing' = 'Internal');

In [0]:
%sql
ALTER TABLE shipments ALTER COLUMN freight_cost SET TAGS ('data_purpose_manufacturing' = 'SupplyChain','sensitive_type_manufacturing' = 'cost');
ALTER TABLE shipments ALTER COLUMN total_value SET TAGS ('data_purpose_manufacturing' = 'SupplyChain');
ALTER TABLE shipments ALTER COLUMN supplier_id SET TAGS ('data_purpose_manufacturing' = 'Audit');
ALTER TABLE shipments ALTER COLUMN tracking_number SET TAGS ('sensitive_type_manufacturing' = 'serial_number','asset_criticality_manufacturing' = 'Medium');

In [0]:
%sql
ALTER TABLE employee_contacts ALTER COLUMN email SET TAGS ('sensitive_type_manufacturing' = 'email','ip_sensitivity_manufacturing' = 'Internal');
ALTER TABLE employee_contacts ALTER COLUMN phone SET TAGS ('sensitive_type_manufacturing' = 'phone','ip_sensitivity_manufacturing' = 'Internal');
ALTER TABLE employee_contacts ALTER COLUMN full_name SET TAGS ('data_purpose_manufacturing' = 'SupplyChain','sensitive_type_manufacturing' = 'name');

In [0]:
%sql
ALTER TABLE performance_metrics ALTER COLUMN maintenance_cost SET TAGS ('data_purpose_manufacturing' = 'SupplyChain','sensitive_type_manufacturing' = 'cost');
ALTER TABLE performance_metrics ALTER COLUMN energy_kwh SET TAGS ('asset_criticality_manufacturing' = 'High');
ALTER TABLE performance_metrics ALTER COLUMN metric_date SET TAGS ('shift_hours_manufacturing' = 'Day');
ALTER TABLE suppliers ALTER COLUMN supplier_name SET TAGS ('sensitive_type_manufacturing' = 'name','data_purpose_manufacturing' = 'Audit');

In [0]:
%sql
ALTER TABLE assets ALTER COLUMN asset_id SET TAGS ('asset_criticality_manufacturing' = 'High');
ALTER TABLE assets ALTER COLUMN latitude SET TAGS ('sensitive_type_manufacturing' = 'gps');
ALTER TABLE assets ALTER COLUMN longitude SET TAGS ('sensitive_type_manufacturing' = 'gps');

In [0]:
%sql
SELECT '✅ Tags applied successfully to manufacturing tables!' AS status;

## ✅ Success!

Tag policies have been created successfully and governed tags have been assigned!

### What You Just Created:
- ✅ Governed tags for capturing data sensitivity 
- ✅ Tag assignment to tables

### 🎯 Next Step:

Continue to **`4_Test_ABAC_Policies.ipynb`** to define ABAC policies using governed tags and test them on the datasets in the manufacturing schema

---